# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [2]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [3]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [4]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [5]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [6]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources while maintaining a consistent tone and style.
2. **Personalization**: Generative AI can be used to create personalized recommendations for customers based on their behavior, preferences, and demographic data. For example, online retailers can use generative AI to suggest products that are likely to interest a customer based on their browsing history.
3. **Design and Prototyping**: Generative AI can be used to generate designs, prototypes, and mockups for various products such as clothing, furniture, and electronics. This can help businesses speed up the design process and reduce costs associated with traditional design methods.
4. **Marketing Automation**: Generative AI can be used to a

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to create personalized content such as articles, social media posts, and product descriptions, reducing the need for human writers and editors.
2. **Image and Video Generation**: Companies can use generative AI to create high-quality images and videos for marketing materials, e-commerce websites, and social media platforms.
3. **Product Design**: Generative AI can be used to design products such as furniture, clothing, and accessories, reducing the need for physical prototypes and speeding up the product development process.
4. **Chatbots and Virtual Assistants**: Generative AI can be used to create more sophisticated chatbots and virtual assistants that can understand natural language and provide personalized customer support.
5. **Music and Audio Generation**: Generative AI can be used to create music, audio effects, and soundscapes for various in

## Alternative approach - using OpenAI python library to connect to Ollama

In [8]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, leveraging its ability to create new content, improve existing processes, and drive innovation. Some examples include:

1. Content Generation:
   - Automated blog posting for marketing teams
   - Creating product descriptions from text-based data sets
   - Designing advertisements with personalized messaging

2. Image and Audio Creation:
   - Artificial photography for advertising campaigns with highly realistic images
   - Music creation for films and commercials using machine learning algorithms
   - Customization of product designs in e-commerce platforms

3. Speech Generation:
   - Chatbots that provide customer support to assist various departments of the company
   - Personalized voice recordings on marketing materials
   - Synthetic speech synthesis for automated messages

4. Predictive Analytics:
   - Generating sales predictions through AI models trained on historical market data
   - Predicting demand

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [28]:

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import ollama

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
OLLAMA_URL = "http://localhost:11434/v1"
OLLAMA_API_KEY = "ollama"
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


In [29]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [30]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [31]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [32]:

def summarize(url):
    website = Website(url)
    ollama_via_openai = OpenAI(base_url=OLLAMA_URL, api_key=OLLAMA_API_KEY)
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [33]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [34]:
display_summary("https://www19.pelisplushd.to/")

# Pelisplus Website Summary

## Overview

The Pelisplus website is a popular platform that allows users to watch movies and TV series online for free. The website has an extensive catalog of content, including genres such as action, animation, comedy, drama, fantasy, and more.

### Movies and TV Series

The website features a vast collection of movies and TV series in high definition (HD). Users can browse through various filters, including alphabetical order, release year, and genre. The website also allows users to search for specific titles or actors.

### News and Announcements

There are no notable news or announcements on the website homepage. However:

* **Upcoming Releases**: While there is a list of upcoming releases, it appears dated and incomplete.
* **New Content**: There is an ongoing section displaying new content added to the platform recently, but this does not include live updates.

### Website Design and Features

The website's design is straightforward, with easy-to-use navigation. However:

* **Quality Issues**: Some users have reported issues with video quality and buffering.
* **Security Measures**: No evident or notable security features.

### Monetization

At the time of our update, there was no information available on how Pelisplus generates revenue, such as advertising or paid subscriptions.

# Conclusion

The Pelisplus website provides a vast library of content but appears to lack updates and live streaming news. While many users have reported positive experiences with their overall viewing quality and selection options, it is essential for the platform to work effectively in both offering content without significant disruptions and to secure ongoing user engagement through constant updates and improvements.

Stay tuned for any changes in these characteristics as more time goes by!

Disclaimer: Last assessed information regarding security features was insufficient, but a review may exist elsewhere which might assist.

As always: Always prioritize your safety while using online platforms where required.